In [ ]:
#importing the necessary libraries and dependencies
import pandas as pd
import numpy as np
import seaborn as sns;
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from numpy import array
from keras.models import Sequential
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import time

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('val%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('val%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('val%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# loading the data into the dataframe
dataset = pd.read_csv('/content/drive/MyDrive/AQI/Data.csv', header=0, index_col=0)
dataset.head()

In [ ]:
# dataset.rename(columns = {'Unnamed: 0':'ID'}, inplace = True)

In [ ]:
# #number of rows and columns in the dataset
print(dataset.shape)

In [ ]:
dataset=dataset.dropna()

In [ ]:
# #statistical information about columns
print(dataset.describe())

In [ ]:
# df=dataset.describe()
# df.to_excel('/content/drive/MyDrive/AQI/Description.xlsx')

In [ ]:
dataset.drop({'wd',}, axis = 1, inplace = True)
dataset.drop({'station',}, axis = 1, inplace = True)
dataset.drop({'date',}, axis = 1, inplace = True)
dataset.drop({'AQI',}, axis = 1, inplace = True)
# dataset.drop({'dateInt',}, axis = 1, inplace = True)
# dataset.drop({'AQI_numeric',}, axis = 1, inplace = True)

In [ ]:
dataset.head()

In [ ]:
# dataset = pd.DataFrame(dataset, columns=["PM2.5", "PM10", "SO2", "NO2", "CO", "O3", "PRES", "DEWP", "RAIN", "WSPM",])
# # dataset.plot.box()
# dataset.plot( kind="area", figsize=(15, 12))
# plt.show()

In [ ]:
# #checking how many null values are in each column
# dataset.drop({'wd',}, axis = 1, inplace = True)
# dataset.drop({'station',}, axis = 1, inplace = True)
# dataset.drop({'date',}, axis = 1, inplace = True)
# dataset.drop({'AQI',}, axis = 1, inplace = True)
# dataset.drop({'TEMP',}, axis = 1, inplace = True)
# dataset.drop({'PRES',}, axis = 1, inplace = True)
# dataset.drop({'DEWP',}, axis = 1, inplace = True)
# dataset.drop({'RAIN',}, axis = 1, inplace = True)
# dataset.drop({'WSPM',}, axis = 1, inplace = True)
# dataset.drop({'dateInt',}, axis = 1, inplace = True)
# dataset.drop({'AQI_numeric',}, axis = 1, inplace = True)

In [ ]:
# #number of rows and columns in the dataset
print(dataset.shape)

Just doing `df.dropna()` drops all the NaN values only for the current execution of the cell. If you do the above `df.isnull().sum()` now, you can see that null values still persists. You can solve this by assigning the obtained output of
`df.dopna()` to the variable `df` which stores our data (dataframe)

In [ ]:
# correlations = dataset.corr()
# fig, ax = plt.subplots(figsize=(15,15))
# sns.heatmap(correlations, vmax=1.0, center=0, fmt='.2f', square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .70})
# plt.show();

In [ ]:
# g = sns.pairplot(dataset)

In [ ]:
# dataset.plot(kind='density', subplots=True, layout=(4,4), sharex=False, figsize=(30,20))
# plt.show()

In [ ]:
# values = dataset.values
# # integer encode direction
# encoder = LabelEncoder()
# values[:,4] = encoder.fit_transform(values[:,4])
# print(values)
# pd.DataFrame(values).to_excel('/content/drive/MyDrive/AQI/IntergerEncoding.xlsx')

In [ ]:

# # ensure all data is float
# values = values.astype('float32')
# np.savetxt('/content/drive/MyDrive/AQI/my_data.csv', values, delimiter=',')
# # values.to_csv(r'/content/drive/MyDrive/AQI/my_data.csv', index=False)
# print(values)

# pd.DataFrame(values).to_excel('/content/drive/MyDrive/AQI/FloatConversion.xlsx')

In [ ]:
# reframed = series_to_supervised(values, 1, 1)
# # split into train and test sets
# values = reframed.values
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled_features = scaler.fit_transform(values[:,:-1])
# scaled_label = scaler.fit_transform(values[:,-1].reshape(-1,1))
# values = np.column_stack((scaled_features, scaled_label))
# print(values)
# pd.DataFrame(values).to_excel('/content/drive/MyDrive/AQI/MinMaxScaling.xlsx')

In [ ]:
values = dataset.values
# integer encode direction
encoder = LabelEncoder()
values[:,4] = encoder.fit_transform(values[:,4])
# ensure all data is float
values = values.astype('float32')
reframed = series_to_supervised(values, 1, 1)
reframed.drop(reframed.columns[[-7,-6,-5,-4,-3,-2,-1]], axis=1, inplace=True)
print(reframed.head())

# split into train and test sets
values = reframed.values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(values[:,:-1])
scaled_label = scaler.fit_transform(values[:,-1].reshape(-1,1))
values = np.column_stack((scaled_features, scaled_label))


In [ ]:
n_train_hours = 365 * 24 * 40
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]
# split into input and outputs
# features take all values except the var1
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)


In [ ]:
from google.colab import files

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(17,8))
plt.plot(train_y, label='train_y', color='#0096FF',)
plt.plot(test_y, label='test_y',color='#1F51FF',)

plt.legend(['train', 'test'], loc='upper left')

plt.savefig("traintest.pdf")
files.download("traintest.pdf")
#plt.show()

In [ ]:
# model = tf.keras.Sequential([keras.layers.Dense(units=10, input_shape=[2, 1]),
#                              keras.layers.Dense(units=5, input_shape=[1]),
#                              keras.layers.Dense(units=1, input_shape=[1])]) #ANN Part

# model.add(LSTM(128, input_shape=(X_split_train[1], X_split_train[2])))
# model.add(Dense(50, activation='tanh'))
# model.add(Dense(1))
# #keras.optimizers.SGD(lr=0.001, decay=1e-5, momentum=1.0, nesterov=False)
# #keras.optimizers.SGD(lr=0.01, decay=1e-5, momentum=0.9, nesterov=True) #good

# #keras.optimizers.RMSprop(learning_rate=0.01, rho=0.9)

# keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.9, amsgrad=False)
# model.compile(loss='mae', optimizer='adam', metrics=['accuracy'] )
# tf.keras.utils.plot_model(model=model, show_shapes=True)


In [ ]:
# design network

model = tf.keras.Sequential([keras.layers.Dense(units=10, input_shape=[1, 18]),
                             keras.layers.Dense(units=5, input_shape=[1]),
                             keras.layers.Dense(units=1, input_shape=[1])]) #ANN Part

model.add(LSTM(128, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(100, activation='tanh'))
model.add(Dense(1))
#keras.optimizers.SGD(lr=0.001, decay=1e-5, momentum=1.0, nesterov=False)
#keras.optimizers.SGD(lr=0.01, decay=1e-5, momentum=0.9, nesterov=True) #good

#keras.optimizers.RMSprop(learning_rate=0.01, rho=0.9)

keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.9, amsgrad=False)
model.compile(loss='mae', optimizer='adam', metrics=['accuracy'] )
tf.keras.utils.plot_model(model=model, show_shapes=True)
tf.keras.utils.plot_model(model=model, show_shapes=True)

In [ ]:
model.summary()

In [ ]:
start = time.time()

# # fit network
history = model.fit(train_X, train_y, epochs=10, validation_data=(test_X, test_y),
                    verbose=1, shuffle=False)

# plot history
pyplot.figure(figsize=(17,8))
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
end = time.time()
print('This took {} seconds.'.format(end - start))
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
mae = tf.keras.metrics.mean_absolute_error(inv_y, inv_yhat)
inv_y = scaler.inverse_transform(inv_y.reshape(-1,1))
inv_yhat = scaler.inverse_transform(inv_yhat.reshape(-1,1))
from matplotlib import pyplot as plt

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
# calculate matrics
mse = mean_squared_error(test_y, yhat)
mae = mean_absolute_error(test_y, yhat)
mape = mean_absolute_percentage_error(test_y, yhat)


print('MSE: %.5f' % mse)
print('MAE : %.5f' % mae)
print('MAPE : %.5f' % mape)

In [ ]:
mse = mean_squared_error(test_y, yhat)
print('MSE: %.5f' % mse)


In [ ]:
def plot_predicted(predicted_data, true_data):
    fig, ax = plt.subplots(figsize=(17,8))
    ax.set_title('Prediction vs. Actual after 10 epochs of training')
    ax.plot(true_data, label='True Data', color='#0096FF', linewidth='3')

    ax.plot(predicted_data, label='Prediction', color='#1F51FF', linewidth='2')
    plt.legend()
    plt.savefig("Predict.pdf")
    files.download("Predict.pdf")
    plt.show()

plot_predicted(inv_yhat, inv_y)
print('Root Mean Squared Error: {:.4f}'.format(rmse))
print("R2 score : %.2f" % r2_score(inv_y,inv_yhat))

In [ ]:
# summarize history for loss
pyplot.figure(figsize=(17,8))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.savefig("loss.pdf")
files.download("loss.pdf")
plt.show()

In [ ]:
t=history.history['val_loss']
np.average(t)

In [ ]:
plt.plot(yhat)


In [ ]:
plt.plot(test_y)

In [ ]:

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig("Accuracy.pdf")
files.download("Accuracy.pdf")
plt.show()

In [ ]:
sns.jointplot(x=dataset['PM2.5'], y=dataset['PM10'], data = dataset)

In [ ]:
sns.jointplot(x=dataset['PM2.5'], y=dataset['SO2'], data = dataset)

In [ ]:
sns.jointplot(x=dataset['PM2.5'], y=dataset['NO2'], data = dataset)

In [ ]:
sns.jointplot(x=dataset['PM2.5'], y=dataset['CO'], data = dataset)

In [ ]:
sns.jointplot(x=dataset['PM2.5'], y=dataset['O3'], data = dataset)

In [ ]:
# loading the data into the dataframe
df = pd.read_csv('/content/drive/MyDrive/AQI/Data.csv', header=0, index_col=0)
df.head()

In [ ]:
#defining training and testing data
x_train = df[:350400]
y_train = x_train['PM2.5']
x_test = df[350400:418946]
y_test = x_test['PM2.5']
print(y_test)

There are many pollutants. Let's first try to predict PM2.5 concentration values. Let the years 2016 and 2017 be the testing set. As you can see below, these 2 years account for 21.9% of the data (test set)

In [ ]:
df.loc[350400:418946].count() / df.shape[0] * 100

In [ ]:
#Normalizing training data
train_norm = x_train['PM2.5']

#converted into array as all the methods available are for arrays and not lists
train_norm_arr = np.asarray(train_norm)
train_norm = np.reshape(train_norm_arr, (-1, 1))

#Scaling all values between 0 and 1 so that large values don't just dominate
scaler = MinMaxScaler(feature_range=(0, 1))
train_norm = scaler.fit_transform(train_norm)
for i in range(5):
    print(train_norm[i])

Even after normalization and scaing, null values are possible (many people disregard this). Let's check if any null values are present.

In [ ]:
count = 0
for i in range(len(train_norm)):
    if train_norm[i] == 0:
        count = count +1
print('Number of null values in train_norm = ', count)

In [ ]:
#removing null values
train_norm = train_norm[train_norm!=0]

In [ ]:
#Normalizing testing data and repeating the same process as done for training data
test_norm = x_test['PM2.5']
test_norm_arr = np.asarray(test_norm)
test_norm = np.reshape(test_norm_arr, (-1, 1))
scaler = MinMaxScaler(feature_range=(0, 1))
test_norm = scaler.fit_transform(test_norm)
for i in range(5):
    print(test_norm[i])

In [ ]:
count = 0
for i in range(len(test_norm)):
    if test_norm[i] == 0:
        count = count + 1
print('Number of null values in test_norm = ', count)

In [ ]:
#removing null values
test_norm = test_norm[test_norm != 0]

In [ ]:
print(train_norm.shape)
print(test_norm.shape)

Since this is a time series data, we should be predicting the values after looking at a set of values rather than just a single value like we usually do. This takes into account the correlation between the data points and the timestamps. Because the neighbours should be considered for how the values change over time. Let's define a function to do this.

The below function called split_sequence splits the sequence into sets of n values. This n is given as n_steps (step_size). For example, if n=3, we split the sequence in groups of 3. We create 2 empty lists and append the split sequences.

In [ ]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X),array(y)

Here the number of features = 1 as we will be predicting a single value. Let's reshape the split sequences into the format of number of rows, number of columns. (shape[0], shape[1]). In the output, we can see that groups of 3 since n_steps = 3 have been obtained.

In [ ]:
n_steps = 2
X_split_train, y_split_train = split_sequence(train_norm, n_steps)
#for i in range(len(X_split_train)):
    #print(X_split_train[i], y_split_train[i])
n_features = 1
X_split_train = X_split_train.reshape((X_split_train.shape[0], X_split_train.shape[1], n_features))
for i in range(5):
    print(X_split_train)

In [ ]:
print (X_split_train.shape)

You can see below that, we predict the value for the first 3 values, then consider that output as one of the 3 values in the next set. For example, we preedict 0.1 first, then we take that 0.1 as input in the second set and so on.

In [ ]:
X_split_test, y_split_test = split_sequence(test_norm, n_steps)
for i in range(5):
    print(X_split_test[i], y_split_test[i])
n_features = 1
X_split_test = X_split_test.reshape((X_split_test.shape[0], X_split_test.shape[1], n_features))

In [ ]:
print (X_split_test.shape)

Let's define our neural network (LSTM: Long Short Term Memory). Let's add 50 nodes in our first layer with a ReLU (Rectified linear unit) activation. Their shape will be step size, number of features. Then we will add, a dense layer with one node for the output.

We can try out different optimizers to see which minimizes loss and maximizes accuracy. Stochastic gradient descent (SGD), Adam, AdaBoost, RMSProp are few of them. lr = learning rate, decay = by how much to decay the learning rate, momentum = how much should the gradient descent be accelerated to dampen oscillations, nesterov = whether to use nesterov momentum. Nesterov has stronger convergence for convex functions. And then we compile using MSE (mean squared loss) as our loss function.

In [ ]:
from math import sqrt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import numpy as np
import pandas as pd

In [ ]:
model = tf.keras.Sequential([keras.layers.Dense(units=10, input_shape=[2, 1]),
                             keras.layers.Dense(units=5, input_shape=[1]),
                             keras.layers.Dense(units=1, input_shape=[1])]) #ANN Part

model.add(LSTM(128, input_shape=(X_split_train[1], X_split_train[2])))
model.add(Dense(100, activation='tanh'))
model.add(Dense(1))
#keras.optimizers.SGD(lr=0.001, decay=1e-5, momentum=1.0, nesterov=False)
#keras.optimizers.SGD(lr=0.01, decay=1e-5, momentum=0.9, nesterov=True) #good

#keras.optimizers.RMSprop(learning_rate=0.01, rho=0.9)

keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.9, amsgrad=False)
model.compile(loss='mae', optimizer='adam', metrics=['accuracy'] )
tf.keras.utils.plot_model(model=model, show_shapes=True)
model.summary()



In [ ]:
# fit model
hist = model.fit(X_split_train, y_split_train, validation_data=(X_split_test, y_split_test), epochs=10, verbose = 1)

In [ ]:
print(hist.history.keys())

In [ ]:
yhat = model.predict(X_split_test)
for i in range(5):
    print(yhat[i])

In [ ]:
# mse = mean_squared_error(y_split_test, yhat)
# print('MSE: %.5f' % mse)

In [ ]:
def plot_predicted(predicted_data, true_data):
    fig, ax = plt.subplots(figsize=(17,8))
    ax.set_title('Prediction vs. Actual after 10 epochs of training')
    ax.plot(true_data, label='True Data', color='red', linewidth='3')

    ax.plot(predicted_data, label='Prediction', color='green', linewidth='2')
    plt.legend()
    # plt.savefig("Pre.pdf")
    # files.download("Pre.pdf")
    plt.show()
mse = mean_squared_error(y_split_test, yhat)
print('MSE: %.5f' % mse)
plot_predicted(yhat[:1000,], y_split_test[:1000,])
print('Root Mean Squared Error: {:.4f}'.format(mse))
print("R2 score : %.2f" % r2_score(y_split_test,yhat))


In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
# calculate matrics
mse = mean_squared_error(y_split_test, yhat)
mae = mean_absolute_error(y_split_test, yhat)
mape = mean_absolute_percentage_error(y_split_test, yhat)


print('MSE: %.5f' % mse)
print('MAE : %.5f' % mae)
print('MAPE : %.5f' % mape)

In [ ]:
#defining training and testing data
x_train = df[:335156 ]
y_train = x_train['PM10']
x_test = df[335156:418946]
y_test = x_test['PM10']
print(y_test)
df.loc[335156:418946].count() / df.shape[0] * 100
#Normalizing training data
train_norm = x_train['PM10']

#converted into array as all the methods available are for arrays and not lists
train_norm_arr = np.asarray(train_norm)
train_norm = np.reshape(train_norm_arr, (-1, 1))

#Scaling all values between 0 and 1 so that large values don't just dominate
scaler = MinMaxScaler(feature_range=(0, 1))
train_norm = scaler.fit_transform(train_norm)
for i in range(5):
    print(train_norm[i])

count = 0
for i in range(len(train_norm)):
    if train_norm[i] == 0:
        count = count +1
print('Number of null values in train_norm = ', count)
#removing null values
train_norm = train_norm[train_norm!=0]
#Normalizing testing data and repeating the same process as done for training data
test_norm = x_test['PM10']
test_norm_arr = np.asarray(test_norm)
test_norm = np.reshape(test_norm_arr, (-1, 1))
scaler = MinMaxScaler(feature_range=(0, 1))
test_norm = scaler.fit_transform(test_norm)
for i in range(5):
    print(test_norm[i])

count = 0
for i in range(len(test_norm)):
    if test_norm[i] == 0:
        count = count + 1
print('Number of null values in test_norm = ', count)
#removing null values
test_norm = test_norm[test_norm != 0]
print(train_norm.shape)
print(test_norm.shape)

def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X),array(y)

n_steps = 2
X_split_train, y_split_train = split_sequence(train_norm, n_steps)
#for i in range(len(X_split_train)):
    #print(X_split_train[i], y_split_train[i])
n_features = 1
X_split_train = X_split_train.reshape((X_split_train.shape[0], X_split_train.shape[1], n_features))
for i in range(5):
    print(X_split_train)

print (X_split_train.shape)

X_split_test, y_split_test = split_sequence(test_norm, n_steps)
for i in range(5):
    print(X_split_test[i], y_split_test[i])
n_features = 1
X_split_test = X_split_test.reshape((X_split_test.shape[0], X_split_test.shape[1], n_features))

print (X_split_test.shape)

hist = model.fit(X_split_train, y_split_train, validation_data=(X_split_test, y_split_test), epochs=10, verbose = 1)

print(hist.history.keys())

yhat = model.predict(X_split_test)
for i in range(5):
    print(yhat[i])



In [ ]:
def plot_predicted(predicted_data, true_data):
    fig, ax = plt.subplots(figsize=(17,8))
    ax.set_title('Prediction vs. Actual after 10 epochs of training')
    ax.plot(true_data, label='True Data', color='green', linewidth='3')

    ax.plot(predicted_data, label='Prediction', color='orange', linewidth='2')
    plt.legend()
    plt.savefig("PreAc.pdf")
    files.download("PreAc.pdf")
    plt.show()
mse = mean_squared_error(y_split_test, yhat)
print('MSE: %.5f' % mse)
plot_predicted(yhat[:1000,], y_split_test[:1000,])
print('Root Mean Squared Error: {:.4f}'.format(mse))
print("R2 score : %.2f" % r2_score(y_split_test,yhat))

Below, I have plotted the actual true values (first plot) and preedicted values (second plot). One can visually see that the distribution is almost the same. This says that our predictions are very accurate.

In [ ]:
# plt.plot(yhat)
# plt.plot(y_split_test)
# # summarize history for accuracy
# plt.plot(hist.history['accuracy'])
# plt.plot(hist.history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

In [ ]:
# plt.plot(y_split_test)

In [ ]:
# _, train_acc = model.evaluate(X_split_train, y_split_train, verbose=0)
# _, test_acc = model.evaluate(X_split_test, y_split_test, verbose=0)
# print('Train: %.5f, Test: %.5f' % (train_acc, test_acc))

In [ ]:
# # summarize history for accuracy
# plt.plot(hist.history['accuracy'])
# plt.plot(hist.history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

Above, accuracy increase a lot in the last few epochs. Below, the loss gradually decrease. These are positive signs that our model is doing very good.

In [ ]:
# # summarize history for loss
# plt.plot(hist.history['loss'])
# plt.plot(hist.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

Until now, we just ran our model for prediction of a single pollutant. We have 6 pollutants in our dataset and can make predictions for all of them. So, I have made a function which can be used to predict the other pollutants rather than having to write the code again and again. I have commented the function calls. You can fork this kernel to uncomment and predit the other pollutants (Coz it would take up a lot of space and time).

In [ ]:
def compute(var):
    train_norm = x_train[var]
    train_norm_arr = np.asarray(train_norm)
    train_norm = np.reshape(train_norm_arr, (-1, 1))
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_norm = scaler.fit_transform(train_norm)
    train_norm = train_norm[train_norm != 0]

    test_norm = x_test[var]
    test_norm_arr = np.asarray(test_norm)
    test_norm = np.reshape(test_norm_arr, (-1, 1))
    scaler = MinMaxScaler(feature_range=(0, 1))
    test_norm = scaler.fit_transform(test_norm)
    test_norm = test_norm[test_norm != 0]

    X_split_train, y_split_train = split_sequence(train_norm, n_steps)
    X_split_train = X_split_train.reshape((X_split_train.shape[0], X_split_train.shape[1], n_features))

    X_split_test, y_split_test = split_sequence(test_norm, n_steps)
    X_split_test = X_split_test.reshape((X_split_test.shape[0], X_split_test.shape[1], n_features))

    hist = model.fit(X_split_train, y_split_train, validation_data=(X_split_test, y_split_test), epochs=10, verbose = 1)

    yhat = model.predict(X_split_test)

    mse = mean_squared_error(y_split_test, yhat)
    print('MSE :' ,mse)
    print('Root Mean Squared Error: {:.4f}'.format(mse))
    print("R2 score : %.2f" % r2_score(y_split_test,yhat))
    # calculate matrics
    mse = mean_squared_error(y_split_test, yhat)
    mae = mean_absolute_error(y_split_test, yhat)
    mape = mean_absolute_percentage_error(y_split_test, yhat)


    print('MSE: %.5f' % mse)
    print('MAE : %.5f' % mae)
    print('MAPE : %.5f' % mape)

    # plt.plot(hist.history['accuracy'])
    # plt.plot(hist.history['val_accuracy'])
    # plt.title('model accuracy')
    # plt.ylabel('accuracy')
    # plt.xlabel('epoch')
    # plt.legend(['train', 'test'], loc='upper left')
    # plt.show()

    # plt.plot(hist.history['loss'])
    # plt.plot(hist.history['val_loss'])
    # plt.title('model loss')
    # plt.ylabel('loss')
    # plt.xlabel('epoch')
    # plt.legend(['train', 'test'], loc='upper left')
    # plt.show()



In [ ]:
compute('PM2.5')

In [ ]:
def plot_predicted(predicted_data, true_data):
    fig, ax = plt.subplots(figsize=(17,8))
    ax.set_title('Prediction vs. Actual after 10 epochs of training')
    ax.plot(true_data, label='True Data', color='green', linewidth='3')

    ax.plot(predicted_data, label='Prediction', color='orange', linewidth='2')
    plt.legend()
    plt.savefig("PM2.5.pdf")
    files.download("PM2.5.pdf")
    plt.show()
plot_predicted(yhat[:1000,], y_split_test[:1000,])

In [ ]:
compute('PM10')

In [ ]:
def plot_predicted(predicted_data, true_data):
    fig, ax = plt.subplots(figsize=(17,8))
    ax.set_title('Prediction vs. Actual after 10 epochs of training')
    ax.plot(true_data, label='True Data', color='green', linewidth='3')

    ax.plot(predicted_data, label='Prediction', color='orange', linewidth='2')
    plt.legend()
    plt.savefig("PM10.pdf")
    files.download("PM10.pdf")
    plt.show()
plot_predicted(yhat[:1000,], y_split_test[:1000,])


In [ ]:
compute('SO2')

In [ ]:
def plot_predicted(predicted_data, true_data):
    fig, ax = plt.subplots(figsize=(17,8))
    ax.set_title('Prediction vs. Actual after 10 epochs of training')
    ax.plot(true_data, label='True Data', color='green', linewidth='3')

    ax.plot(predicted_data, label='Prediction', color='orange', linewidth='2')
    plt.legend()
    plt.savefig("SO2.pdf")
    files.download("SO2.pdf")
    plt.show()
plot_predicted(yhat[:1000,], y_split_test[:1000,])

In [ ]:
compute('NO2')

In [ ]:
def plot_predicted(predicted_data, true_data):
    fig, ax = plt.subplots(figsize=(17,8))
    ax.set_title('Prediction vs. Actual after 10 epochs of training')
    ax.plot(true_data, label='True Data', color='green', linewidth='3')

    ax.plot(predicted_data, label='Prediction', color='orange', linewidth='2')
    plt.legend()
    plt.savefig("NO2.pdf")
    files.download("NO2.pdf")
    plt.show()
plot_predicted(yhat[:1000,], y_split_test[:1000,])

In [ ]:
compute('CO')

In [ ]:
def plot_predicted(predicted_data, true_data):
    fig, ax = plt.subplots(figsize=(17,8))
    ax.set_title('Prediction vs. Actual after 10 epochs of training')
    ax.plot(true_data, label='True Data', color='green', linewidth='3')

    ax.plot(predicted_data, label='Prediction', color='orange', linewidth='2')
    plt.legend()
    plt.savefig("CO.pdf")
    files.download("CO.pdf")
    plt.show()
plot_predicted(yhat[:1000,], y_split_test[:1000,])


In [ ]:
compute('O3')

In [ ]:
def plot_predicted(predicted_data, true_data):
    fig, ax = plt.subplots(figsize=(17,8))
    ax.set_title('Prediction vs. Actual after 10 epochs of training')
    ax.plot(true_data, label='True Data', color='green', linewidth='3')

    ax.plot(predicted_data, label='Prediction', color='orange', linewidth='2')
    plt.legend()
    plt.savefig("O3.pdf")
    files.download("O3.pdf")
    plt.show()
plot_predicted(yhat[:1000,], y_split_test[:1000,])